In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import warnings
from langchain_core._api import LangChainBetaWarning
from rag_engine.retriever import build_retriever
from rag_engine.chains import get_rag_chain

warnings.filterwarnings("ignore", category=LangChainBetaWarning)
load_dotenv()
retriever = build_retriever()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_chain = get_rag_chain(llm, retriever)

In [3]:
for chunk in rag_chain.stream({"question": "What professors are doing research in robotics?"}):
    print(chunk, end="", flush=True)

Based on the provided context, here are the professors at Northwestern University who are conducting research in robotics:

1. **Matthew Elwin**
   - **Department**: Mechanical Engineering
   - **Position**: Associate Professor of Instruction of Mechanical Engineering; Director of the Master of Science in Robotics Program
   - **Research Areas**: Swarm robotics, cooperative robot manipulation and dexterity, robotic navigation, and mechatronics.
   - **Contact**: 2145 Sheridan Road, Evanston, IL 60208-3109; 847-491-5474; [elwin@northwestern.edu](mailto:elwin@northwestern.edu)
   - **Website**: [Center for Robotics and Biosystems](https://robotics.northwestern.edu/)

2. **Kevin Lynch**
   - **Department**: Mechanical Engineering
   - **Position**: Professor of Mechanical Engineering; Director, Center for Robotics and Biosystems
   - **Research Areas**: Interdisciplinary fundamentals of interfacing sensors and actuators with computers, robotic manipulation, and mechatronics.
   - **Contac